In [11]:
import os
import pandas as pd
import numpy as np

## Extracting missing dataset statistics from v4 subset - 4.6.22

In [3]:
dataset_v4 = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/data/processed/output_of_preprocessing/wikidata5M_human_facts_subset_060522_v4.tsv',
                         sep = '\t', names = ['head_entity', 'relation', 'tail_entity'])
dataset_v4.head()

,head_entity,relation,tail_entity
0,Q18924322,P1344,Q11886402
1,Q4274840,P735,Q4927524
2,Q6124446,P54,Q1186020
3,Q3058238,P136,Q76092
4,Q4353413,P106,Q488111


In [7]:
set_of_head_entities = set(dataset_v4['head_entity'])
set_of_tail_entities = set(dataset_v4['tail_entity'])
set_of_all_QIDs = set_of_head_entities.union(set_of_tail_entities)
print(f'Number of unique head entities (= human entities): {len(set_of_head_entities)}')
print(f'Number of unique tail entities: {len(set_of_tail_entities)}')
print(f'Total number of entities: {(len(set_of_all_QIDs))}')
set_of_all_PIDs = set(dataset_v4['relation'])
print(f'Total number of relations: {(len(set_of_all_PIDs))}')
print(f'Total number of triples: {(len(dataset_v4))}')

Number of unique head entities (= human entities): 1269907
Number of unique tail entities: 177143
Total number of entities: 1396220
Total number of relations: 225
Total number of triples: 3101160


## Calculate tail accuracy metrics for the KG only FB15k-237 models based on the result sheets - 4.6.22

In [8]:
path_to_results = '/home/lena/git/master_thesis_bias_in_NLP/results/KG_only/Rossi_models_FB15K-237/'

### TransE

In [10]:
raw_file_Rossi_FB15k237 = pd.read_csv(os.path.join(path_to_results, 'TransE/transe_filtered_ranks.csv'),
                                      sep = ';',
                                      names = ['head_id', 'relation_id', 'true_tail_id',
                                               'rank_head', 'rank_tail'])
raw_file_Rossi_FB15k237.head()

,head_id,relation_id,true_tail_id,rank_head,rank_tail
0,/m/08966,/travel/travel_destination/climate./travel/tra...,/m/05lf_,1,1
1,/m/01hww_,/music/performance_role/regular_performances./...,/m/01q99h,17,48
2,/m/09v3jyg,/film/film/release_date_s./film/film_regional_...,/m/0f8l9c,13,2
3,/m/02jx1,/location/location/contains,/m/013t85,2,1290
4,/m/02jx1,/location/location/contains,/m/0m0bj,2,114


In [18]:
mean_rank_head = raw_file_Rossi_FB15k237['rank_head'].mean()
mean_rank_tail = raw_file_Rossi_FB15k237['rank_tail'].mean()
mean_rank_both = np.mean([raw_file_Rossi_FB15k237['rank_head'], raw_file_Rossi_FB15k237['rank_tail']])
print(f'mean rank head: {mean_rank_head}')
print(f'mean rank tail: {mean_rank_tail}')
print(f'mean rank both: {mean_rank_both}')

mean rank head: 281.0699187787455
mean rank tail: 136.74904589490166
mean rank both: 208.90948233682357


In [23]:
MRR_both = np.mean([raw_file_Rossi_FB15k237['rank_head'].apply(lambda row: 1/row), raw_file_Rossi_FB15k237['rank_tail'].apply(lambda row: 1/row)])
MRR_head = np.mean(raw_file_Rossi_FB15k237['rank_head'].apply(lambda row: 1/row))
MRR_tail = np.mean(raw_file_Rossi_FB15k237['rank_tail'].apply(lambda row: 1/row))
print(f'MRR_both:{MRR_both}')
print(f'MRR_head:{MRR_head}')
print(f'MRR_tail:{MRR_tail}')

MRR_both:0.30988224890994276
MRR_head:0.20999301671029683
MRR_tail:0.4097714811095887


In [24]:
total_number_of_triples = len(raw_file_Rossi_FB15k237)
total_number_of_triples

20438

In [38]:
k = 10
hits_at_k_head = len(raw_file_Rossi_FB15k237[raw_file_Rossi_FB15k237['rank_head'] <= k])/total_number_of_triples
hits_at_k_tail = len(raw_file_Rossi_FB15k237[raw_file_Rossi_FB15k237['rank_tail'] <= k])/total_number_of_triples
hits_at_k_both = np.mean([hits_at_k_head, hits_at_k_tail])
print(f'hits_at_k_head:{hits_at_k_head}')
print(f'hits_at_k_tail:{hits_at_k_tail}')
print(f'hits_at_k_both:{hits_at_k_both}')

hits_at_k_head:0.382033467071142
hits_at_k_tail:0.6110186906742343
hits_at_k_both:0.4965260788726882


### DistMult

In [54]:
raw_file_Rossi_FB15k237 = pd.read_csv(os.path.join(path_to_results, 'DistMult/distmult_filtered_ranks.csv'),
                                      sep = ';',
                                      names = ['head_id', 'relation_id', 'true_tail_id',
                                               'rank_head', 'rank_tail'])
raw_file_Rossi_FB15k237.head()

,head_id,relation_id,true_tail_id,rank_head,rank_tail
0,/m/08966,/travel/travel_destination/climate./travel/tra...,/m/05lf_,1,1
1,/m/01hww_,/music/performance_role/regular_performances./...,/m/01q99h,50,71
2,/m/09v3jyg,/film/film/release_date_s./film/film_regional_...,/m/0f8l9c,20,4
3,/m/02jx1,/location/location/contains,/m/013t85,1,320
4,/m/02jx1,/location/location/contains,/m/0m0bj,1,120


In [55]:
mean_rank_head = raw_file_Rossi_FB15k237['rank_head'].mean()
mean_rank_tail = raw_file_Rossi_FB15k237['rank_tail'].mean()
mean_rank_both = np.mean([raw_file_Rossi_FB15k237['rank_head'], raw_file_Rossi_FB15k237['rank_tail']])
print(f'mean rank head: {mean_rank_head}')
print(f'mean rank tail: {mean_rank_tail}')
print(f'mean rank both: {mean_rank_both}')

mean rank head: 238.25256874449553
mean rank tail: 160.105196203151
mean rank both: 199.17888247382328


In [56]:
MRR_both = np.mean([raw_file_Rossi_FB15k237['rank_head'].apply(lambda row: 1/row), raw_file_Rossi_FB15k237['rank_tail'].apply(lambda row: 1/row)])
MRR_head = np.mean(raw_file_Rossi_FB15k237['rank_head'].apply(lambda row: 1/row))
MRR_tail = np.mean(raw_file_Rossi_FB15k237['rank_tail'].apply(lambda row: 1/row))
print(f'MRR_both:{MRR_both}')
print(f'MRR_head:{MRR_head}')
print(f'MRR_tail:{MRR_tail}')

MRR_both:0.3130524127672369
MRR_head:0.21916327439549604
MRR_tail:0.4069415511389777


In [57]:
total_number_of_triples = len(raw_file_Rossi_FB15k237)
total_number_of_triples

20438

In [58]:
k = 10
hits_at_k_head = len(raw_file_Rossi_FB15k237[raw_file_Rossi_FB15k237['rank_head'] <= k])/total_number_of_triples
hits_at_k_tail = len(raw_file_Rossi_FB15k237[raw_file_Rossi_FB15k237['rank_tail'] <= k])/total_number_of_triples
hits_at_k_both = np.mean([hits_at_k_head, hits_at_k_tail])
print(f'hits_at_{k}_head:{hits_at_k_head}')
print(f'hits_at_{k}_tail:{hits_at_k_tail}')
print(f'hits_at_{k}_both:{hits_at_k_both}')

hits_at_10_head:0.381690967805069
hits_at_10_tail:0.5984440747626969
hits_at_10_both:0.490067521283883


### RotatE

In [59]:
raw_file_Rossi_FB15k237 = pd.read_csv(os.path.join(path_to_results, 'RotatE/rotate_filtered_ranks.csv'),
                                      sep = ';',
                                      names = ['head_id', 'relation_id', 'true_tail_id',
                                               'rank_head', 'rank_tail'])
raw_file_Rossi_FB15k237.head()

,head_id,relation_id,true_tail_id,rank_head,rank_tail
0,/m/08966,/travel/travel_destination/climate./travel/tra...,/m/05lf_,1,1
1,/m/01hww_,/music/performance_role/regular_performances./...,/m/01q99h,41,39
2,/m/09v3jyg,/film/film/release_date_s./film/film_regional_...,/m/0f8l9c,10,2
3,/m/02jx1,/location/location/contains,/m/013t85,4,2330
4,/m/02jx1,/location/location/contains,/m/0m0bj,1,172


In [60]:
mean_rank_head = raw_file_Rossi_FB15k237['rank_head'].mean()
mean_rank_tail = raw_file_Rossi_FB15k237['rank_tail'].mean()
mean_rank_both = np.mean([raw_file_Rossi_FB15k237['rank_head'], raw_file_Rossi_FB15k237['rank_tail']])
print(f'mean rank head: {mean_rank_head}')
print(f'mean rank tail: {mean_rank_tail}')
print(f'mean rank both: {mean_rank_both}')

mean rank head: 244.54993648001565
mean rank tail: 111.66959835825271
mean rank both: 178.1097674191342


In [61]:
MRR_both = np.mean([raw_file_Rossi_FB15k237['rank_head'].apply(lambda row: 1/row), raw_file_Rossi_FB15k237['rank_tail'].apply(lambda row: 1/row)])
MRR_head = np.mean(raw_file_Rossi_FB15k237['rank_head'].apply(lambda row: 1/row))
MRR_tail = np.mean(raw_file_Rossi_FB15k237['rank_tail'].apply(lambda row: 1/row))
print(f'MRR_both:{MRR_both}')
print(f'MRR_head:{MRR_head}')
print(f'MRR_tail:{MRR_tail}')

MRR_both:0.33551477095330423
MRR_head:0.2388598016921442
MRR_tail:0.43216974021446436


In [62]:
total_number_of_triples = len(raw_file_Rossi_FB15k237)
total_number_of_triples

20466

In [65]:
k = 10
hits_at_k_head = len(raw_file_Rossi_FB15k237[raw_file_Rossi_FB15k237['rank_head'] <= k])/total_number_of_triples
hits_at_k_tail = len(raw_file_Rossi_FB15k237[raw_file_Rossi_FB15k237['rank_tail'] <= k])/total_number_of_triples
hits_at_k_both = np.mean([hits_at_k_head, hits_at_k_tail])
print(f'hits_at_{k}_head:{hits_at_k_head}')
print(f'hits_at_{k}_tail:{hits_at_k_tail}')
print(f'hits_at_{k}_both:{hits_at_k_both}')

hits_at_10_head:0.42343398807778754
hits_at_10_tail:0.6377406430176878
hits_at_10_both:0.5305873155477376


## Calculate KG-BERT accuracy on the occupation predictions

In [66]:
KGBERT_predictions = pd.read_csv('/home/lena/git/master_thesis_bias_in_NLP/results/KG_and_LM/KG-BERT/11.05.2022_16:00_train_and_test_default_FB15K-237_6A100_final/ranks_with_metadata_test_merged_only_gender_occupation_29052022.tsv',
                                 sep = '\t')
KGBERT_predictions.head()

,head,relation,tail,rank_head,rank_tail,top_k_scores_head,top_k_entities_IDs_head,top_k_scores_tail,top_k_entities_IDs_tail
0,/m/02ghq,/people/person/profession,/m/01d_h8,2736,253,"[0.0071469075, 0.0069746017, 0.0048356955, 0.0...","['/m/079kdz', '/m/03vrnh', '/m/0d0l91', '/m/03...","[0.026875637, 0.026580634, 0.024800703, 0.0218...","['/m/028fjr', '/m/0kyk', '/m/0d8qb', '/m/0hz28..."
1,/m/01wkmgb,/people/person/profession,/m/09jwl,47,8,"[0.008710656, 0.0054885643, 0.0050288104, 0.00...","['/m/01x4sb', '/m/01r9md', '/m/01v3vp', '/m/01...","[0.028821947, 0.027155459, 0.021254253, 0.0208...","['/m/064xm0', '/m/03bx0bm', '/m/02dsz', '/m/04..."
2,/m/06sy4c,/people/person/gender,/m/05zppz,1,1,"[0.023514604, 0.019892035, 0.01938399, 0.01869...","['/m/06sy4c', '/m/0dn7v', '/m/01wbgdv', '/m/06...","[0.024035277, 0.00085558416, 0.00070633873, 0....","['/m/05zppz', '/m/0d7wh', '/m/02sdk9v', '/m/0g..."
3,/m/0mm1q,/people/person/profession,/m/01d_h8,32,3,"[0.007131098, 0.006959173, 0.0048249983, 0.003...","['/m/079kdz', '/m/03vrnh', '/m/0d0l91', '/m/03...","[0.023830311, 0.022861863, 0.019964686, 0.0122...","['/m/02r96rf', '/m/064xm0', '/m/01d_h8', '/m/0..."
4,/m/03j90,/people/person/profession,/m/05z96,34,2,"[0.0073894546, 0.007005059, 0.0063762814, 0.00...","['/m/02wh0', '/m/0m93', '/m/084w8', '/m/02qvhb...","[0.05024387, 0.040318903, 0.038028486, 0.03464...","['/m/04t_mf', '/m/05z96', '/m/03j90', '/m/0308..."


In [68]:
KGBERT_predictions_only_occupation = KGBERT_predictions[KGBERT_predictions['relation'] == '/people/person/profession']
KGBERT_predictions_only_occupation

,head,relation,tail,rank_head,rank_tail,top_k_scores_head,top_k_entities_IDs_head,top_k_scores_tail,top_k_entities_IDs_tail
0,/m/02ghq,/people/person/profession,/m/01d_h8,2736,253,"[0.0071469075, 0.0069746017, 0.0048356955, 0.0...","['/m/079kdz', '/m/03vrnh', '/m/0d0l91', '/m/03...","[0.026875637, 0.026580634, 0.024800703, 0.0218...","['/m/028fjr', '/m/0kyk', '/m/0d8qb', '/m/0hz28..."
1,/m/01wkmgb,/people/person/profession,/m/09jwl,47,8,"[0.008710656, 0.0054885643, 0.0050288104, 0.00...","['/m/01x4sb', '/m/01r9md', '/m/01v3vp', '/m/01...","[0.028821947, 0.027155459, 0.021254253, 0.0208...","['/m/064xm0', '/m/03bx0bm', '/m/02dsz', '/m/04..."
3,/m/0mm1q,/people/person/profession,/m/01d_h8,32,3,"[0.007131098, 0.006959173, 0.0048249983, 0.003...","['/m/079kdz', '/m/03vrnh', '/m/0d0l91', '/m/03...","[0.023830311, 0.022861863, 0.019964686, 0.0122...","['/m/02r96rf', '/m/064xm0', '/m/01d_h8', '/m/0..."
4,/m/03j90,/people/person/profession,/m/05z96,34,2,"[0.0073894546, 0.007005059, 0.0063762814, 0.00...","['/m/02wh0', '/m/0m93', '/m/084w8', '/m/02qvhb...","[0.05024387, 0.040318903, 0.038028486, 0.03464...","['/m/04t_mf', '/m/05z96', '/m/03j90', '/m/0308..."
5,/m/02vwckw,/people/person/profession,/m/0n1h,117,3,"[0.0044786357, 0.0044699623, 0.0042721443, 0.0...","['/m/03xds', '/m/0hgqq', '/m/071xj', '/m/01wj5...","[0.08184718, 0.033018593, 0.020987144, 0.01971...","['/m/02dsz', '/m/0dz3r', '/m/0n1h', '/m/04j5jl..."
...,...,...,...,...,...,...,...,...,...
1737,/m/025vw4t,/people/person/profession,/m/02krf9,1,1,"[0.007574797, 0.007337841, 0.006771566, 0.0050...","['/m/025vw4t', '/m/01z0lb', '/m/017r13', '/m/0...","[0.08864466, 0.06141172, 0.059917174, 0.051188...","['/m/02krf9', '/m/040p_q', '/m/0ckd1', '/m/0cb..."
1738,/m/059j1m,/people/person/profession,/m/018gz8,4,1,"[0.009188846, 0.0087780105, 0.00871591, 0.0086...","['/m/0lx2l', '/m/02yplc', '/m/05p606', '/m/059...","[0.058022846, 0.04295462, 0.040730957, 0.03579...","['/m/018gz8', '/m/01lhf', '/m/0747nrk', '/m/0n..."
1739,/m/016sp_,/people/person/profession,/m/016z4k,245,13,"[0.005739699, 0.005127326, 0.00485301, 0.00482...","['/m/049qx', '/m/01xzb6', '/m/0fhxv', '/m/01vz...","[0.027426446, 0.02445879, 0.022692293, 0.02189...","['/m/04t_mf', '/m/067nv', '/m/025t7ly', '/m/0g..."
1740,/m/016tb7,/people/person/profession,/m/03gjzk,165,3,"[0.0070127645, 0.006654296, 0.0061596218, 0.00...","['/m/047q2wc', '/m/0988cp', '/m/02bvt', '/m/04...","[0.036629096, 0.028383244, 0.017234819, 0.0168...","['/m/04jjy', '/m/0mbx4', '/m/03gjzk', '/m/0ckd..."


In [69]:
mean_rank_head = KGBERT_predictions_only_occupation['rank_head'].mean()
mean_rank_tail = KGBERT_predictions_only_occupation['rank_tail'].mean()
mean_rank_both = np.mean([KGBERT_predictions_only_occupation['rank_head'], KGBERT_predictions_only_occupation['rank_tail']])
print(f'mean rank head: {mean_rank_head}')
print(f'mean rank tail: {mean_rank_tail}')
print(f'mean rank both: {mean_rank_both}')

mean rank head: 267.2784134248665
mean rank tail: 22.696414950419527
mean rank both: 144.98741418764303


In [70]:
MRR_both = np.mean([KGBERT_predictions_only_occupation['rank_head'].apply(lambda row: 1/row), KGBERT_predictions_only_occupation['rank_tail'].apply(lambda row: 1/row)])
MRR_head = np.mean(KGBERT_predictions_only_occupation['rank_head'].apply(lambda row: 1/row))
MRR_tail = np.mean(KGBERT_predictions_only_occupation['rank_tail'].apply(lambda row: 1/row))
print(f'MRR_both:{MRR_both}')
print(f'MRR_head:{MRR_head}')
print(f'MRR_tail:{MRR_tail}')

MRR_both:0.1485137371219073
MRR_head:0.045770318509529284
MRR_tail:0.25125715573428525


In [71]:
total_number_of_triples = len(KGBERT_predictions_only_occupation)
total_number_of_triples

1311

In [75]:
k = 10
hits_at_k_head = len(KGBERT_predictions_only_occupation[KGBERT_predictions_only_occupation['rank_head'] <= k])/total_number_of_triples
hits_at_k_tail = len(KGBERT_predictions_only_occupation[KGBERT_predictions_only_occupation['rank_tail'] <= k])/total_number_of_triples
hits_at_k_both = np.mean([hits_at_k_head, hits_at_k_tail])
print(f'hits_at_{k}_head:{hits_at_k_head}')
print(f'hits_at_{k}_tail:{hits_at_k_tail}')
print(f'hits_at_{k}_both:{hits_at_k_both}')

hits_at_10_head:0.11136536994660565
hits_at_10_tail:0.5537757437070938
hits_at_10_both:0.3325705568268497
